#### Import libraries

In [61]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Energy datasets

#### Read data

In [62]:
df_energy = pd.read_csv("energia_202301-202304.csv", na_filter=False)

#### 'Hora' column to int

In [63]:
df_energy['Hora'] = df_energy['Hora'].astype(int)

## Meteo datasets

#### Read data

In [64]:
df_meteo = pd.read_csv("meteo_202301-202304.csv", na_filter=False)

#### Transforming 'dt_iso' column to 'Date' and 'Hora' columns

In [65]:
df_meteo['dt_iso'] = pd.to_datetime(df_meteo['dt_iso'].str.replace(' UTC', ''), format='%Y-%m-%d %H:%M:%S %z')
df_meteo['Data'] = df_meteo['dt_iso'].dt.strftime('%Y-%m-%d')
df_meteo['Hora'] = df_meteo['dt_iso'].dt.strftime('%H')
df_meteo['Hora'] = df_meteo['Hora'].astype(int)

#### Drop 'dt_iso' column

In [66]:
df_meteo = df_meteo.drop('dt_iso',axis = 1)

## API dataset

#### Read data

In [67]:
df_api = pd.read_csv("api.csv", na_filter=False)
df_weather = pd.read_csv("new_weather.csv", na_filter=False)

In [68]:
df_weather.drop(columns=['dew_point_2m (°C)'], inplace=True)
df_weather.drop(columns=['rain (mm)'], inplace=True)
df_weather.drop(columns=['et0_fao_evapotranspiration (mm)'], inplace=True)
df_weather.drop(columns=['vapour_pressure_deficit (kPa)'], inplace=True)
df_weather.drop(columns=['soil_temperature_7_to_28cm (°C)'], inplace=True)
df_weather.drop(columns=['soil_temperature_28_to_100cm (°C)'], inplace=True)
df_weather.drop(columns=['soil_temperature_100_to_255cm (°C)'], inplace=True)

#### 'time' column into 'Data' and 'Hora' columns

In [69]:
df_api['time'] = pd.to_datetime(df_api['time'])

df_api['Data'] = df_api['time'].dt.strftime('%Y-%m-%d')
df_api['Hora'] = df_api['time'].dt.strftime('%H')
df_api['Hora'] = df_api['Hora'].astype(int)

df_api.drop(columns=['time'], inplace=True)

#-----------------------------------------

df_weather['time'] = pd.to_datetime(df_weather['time'])

df_weather['Data'] = df_weather['time'].dt.strftime('%Y-%m-%d')
df_weather['Hora'] = df_weather['time'].dt.strftime('%H')
df_weather['Hora'] = df_weather['Hora'].astype(int)

df_weather.drop(columns=['time'], inplace=True)

#### Columns renaming

In [70]:
df_api.columns = [col[:col.rfind(' ')] if col.endswith(')') else col for col in df_api.columns]

df_api.rename(columns={
    "pressure_msl": "sealevel_pressure"
}, inplace=True)

df_weather.columns = [col[:col.rfind(' ')] if col.endswith(')') else col for col in df_weather.columns]

## Merge datasets by 'Data' and 'Hora' columns

In [71]:
merged_df1 = pd.merge(df_energy, df_meteo, on=['Data','Hora'], how='inner')
merged_df2 = pd.merge(merged_df1, df_api, on=['Data','Hora'], how='inner')
df = pd.merge(merged_df2, df_weather, on=['Data','Hora'], how='inner')

## Columns renaming

In [72]:
df.rename(columns={
    'Data':'date',
    'Hora':'hour',
    'Normal (kWh)':'normal',
    'Horário Económico (kWh)':'economic_schedule',
    'Autoconsumo (kWh)':'self-consumption',
    'Injeção na rede (kWh)':'injection'
}, inplace=True)

## Write new csv


In [73]:
df.to_csv('../test.csv', index=False)